<!--NAVIGATION-->
< [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Contents](Index.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) >

# Handling Missing Data

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous.
In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python.
Here and throughout the book, we'll refer to missing data in general as *null*, *NaN*, or *NA* values.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame.
Generally, they revolve around one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions.
For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

## Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy.
While R contains four basic data types, NumPy supports *far* more than this: for example, while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions, signedness, and endianness of the encoding.
Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad."
Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
vals1.dtype

dtype('O')

In [4]:
vals1.dtype.name

'object'

In [5]:
V = np.arange(12).reshape(3,4)
V

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [6]:
V.dtype

dtype('int32')

In [7]:
V[0][3]

3

In [8]:
V[0][3]=None
V

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
%timeit  # it is Used in Jupyter to Check the time take Process for this Current Cell
try:
    a = "H2"
    print(int(a))
except NameError as ne:
    print(ne)

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [ ]:
np.arange(1E6)

In [ ]:
for dtype in ['object','int','float','complex']:
    print("dtype =", dtype)
    %timeit np.arange(1E3, dtype=dtype).sum()
    print()

The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [ ]:
vals1.sum()

In [ ]:
vals1.mean()

In [ ]:
vals1+10

This reflects the fact that addition between an integer and ``None`` is undefined.

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [ ]:
vals2 = np.array([1,3, 4,np.nan]) 
vals2.dtype

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [ ]:
1 + np.nan

In [ ]:
0 *  np.nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [ ]:
vals2.sum(), vals2.min(), vals2.max(),vals2.mean()

NumPy does provide some special aggregations that will ignore these missing values:

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2),np.nanmean(vals2)

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [9]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.
For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA:

In [10]:
x = pd.Series(range(4), dtype=int)
x

0    0
1    1
2    2
3    3
dtype: int32

In [11]:
x[3] = None
x

0    0.0
1    1.0
2    2.0
3    NaN
dtype: float64

In [12]:
x.add(2)  # it will Not Effect the orginap data

0    2.0
1    3.0
2    4.0
3    NaN
dtype: float64

In [13]:
np.nan_to_num(x)

array([0., 1., 2., 0.])

In [14]:
np.nanmax(x)

2.0

In [15]:
np.nanmin(x)

0.0

In [16]:
np.nanmean(x),np.nanmedian(x),np.nancumsum(x)

(1.0, 1.0, array([0., 1., 3., 3.]))

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

In [17]:
x = np.array([10,20,30, np.nan,None])
x

array([10, 20, 30, nan, None], dtype=object)

In [18]:
x.min()

TypeError: '<=' not supported between instances of 'float' and 'NoneType'

In [19]:
x.dtype

dtype('O')

In [20]:
Data = pd.Series([10,20,30, np.nan,None])
Data

0    10.0
1    20.0
2    30.0
3     NaN
4     NaN
dtype: float64

In [21]:
Data.min()

10.0

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull() or isna``: Generate a boolean mask indicating missing values
- ``notnull() or notna``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [22]:
data = pd.Series([1, np.nan, 10.5, None])
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [23]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [24]:
data.isna()

0    False
1     True
2    False
3     True
dtype: bool

In [25]:
data.notna() # where You don't have a Null Values

0     True
1    False
2     True
3    False
dtype: bool

In [26]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [27]:
Df = pd.DataFrame(data,columns=['Data'])
Df

,Data
0,1.0
1,NaN
2,10.5
3,NaN


In [28]:
Df.isnull()

,Data
0,False
1,True
2,False
3,True


In [29]:
Df.isnull().sum()

Data    2
dtype: int64

In [30]:
Df['Age']=np.array([24,23,45,56])
Df

,Data,Age
0,1.0,24
1,NaN,23
2,10.5,45
3,NaN,56


In [31]:
Df['Gender'] = pd.Categorical(['M','F','M','F'])

In [32]:
Df

,Data,Age,Gender
0,1.0,24,M
1,NaN,23,F
2,10.5,45,M
3,NaN,56,F


In [33]:
Df.dtypes

Data       float64
Age          int32
Gender    category
dtype: object

In [34]:
Df.isnull()

,Data,Age,Gender
0,False,False,False
1,True,False,False
2,False,False,False
3,True,False,False


As mentioned in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb), Boolean masks can be used directly as a ``Series`` or ``DataFrame`` index:

In [35]:
Df.isnull().sum()

Data      2
Age       0
Gender    0
dtype: int64

In [36]:
Df.notnull()

,Data,Age,Gender
0,True,True,True
1,False,True,True
2,True,True,True
3,False,True,True


In [37]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [38]:
data[data.notnull()]

0     1.0
2    10.5
dtype: float64

In [39]:
Df[Df.notnull()] # with Null Values

,Data,Age,Gender
0,1.0,24,M
1,NaN,23,F
2,10.5,45,M
3,NaN,56,F


In [40]:
Df[Df['Data'].notnull()]  # with out Null Values

,Data,Age,Gender
0,1.0,24,M
2,10.5,45,M


The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()``
(which removes NA values) and ``fillna()`` (which fills in NA values). For a ``Series``,
the result is straightforward:

In [41]:
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [42]:
data.dropna() # it will Remove all Null Values but it will not effect the original data

0     1.0
2    10.5
dtype: float64

In [43]:
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [44]:
data.fillna(value=2)

0     1.0
1     2.0
2    10.5
3     2.0
dtype: float64

In [45]:
data.fillna(value={1:2.5,3:50})

0     1.0
1     2.5
2    10.5
3    50.0
dtype: float64

In [46]:
mean = np.nanmean(data)
print(mean)

5.75


In [47]:
data.fillna(value=mean)

0     1.00
1     5.75
2    10.50
3     5.75
dtype: float64

In [48]:
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [49]:
fill = np.nanmedian(data)
fill

5.75

In [50]:
data.fillna(value=mean)

0     1.00
1     5.75
2    10.50
3     5.75
dtype: float64

For a ``DataFrame``, there are more options.
Consider the following ``DataFrame``:

In [51]:
df = pd.DataFrame([[1,      np.nan, 2, None],
                   [2,      3,      5, 10  ],
                   [np.nan, 4,      9, 155 ]],columns=['A','B','C','D'])
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,10.0
2,NaN,4.0,9,155.0


We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options for a ``DataFrame``.

By default, ``dropna()`` will drop all rows in which *any* null value is present:

In [52]:
df.dropna() # by Default you get row

,A,B,C,D
1,2.0,3.0,5,10.0


In [53]:
df.dropna(axis=1) # we will get col

,C
0,2
1,5
2,9


Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:

In [54]:
df.dropna(axis='columns')

,C
0,2
1,5
2,9


In [55]:
df.dropna(axis='rows')

,A,B,C,D
1,2.0,3.0,5,10.0


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [56]:
df['D'] = np.nan
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [57]:
df.dropna(axis='columns', how='all')

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,9


In [58]:
df.dropna(axis=1, how='all')

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,9


In [59]:
df.dropna(axis='columns', how='any') # any null value in the null

,C
0,2
1,5
2,9


In [60]:
df.dropna(axis=0, how='any') #row wise

,A,B,C,D


For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [61]:
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [62]:
df.dropna(axis='rows', thresh=3)

,A,B,C,D
1,2.0,3.0,5,NaN


In [63]:
df.dropna(axis='rows', thresh=2)

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [64]:
df.dropna(axis='columns', thresh=3)

,C
0,2
1,5
2,9


In [65]:
df.dropna(axis='columns', thresh=2)

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,9


In [66]:
Df = pd.DataFrame({'Int':[10,20,30,np.nan,50,60],
                  'Float':[20.5,10.5,5.5,2.5,6.5,np.nan],
                   'Complex':[np.nan,2j+3,np.nan,23j+2,5j+2,np.nan]
                  })
Df

,Int,Float,Complex
0,10.0,20.5,NaN
1,20.0,10.5,3.000000+2.000000j
2,30.0,5.5,NaN
3,NaN,2.5,2.000000+23.000000j
4,50.0,6.5,2.000000+5.000000j
5,60.0,NaN,NaN


In [67]:
Df.isnull()

,Int,Float,Complex
0,False,False,True
1,False,False,False
2,False,False,True
3,True,False,False
4,False,False,False
5,False,True,True


In [68]:
Df.isna()

,Int,Float,Complex
0,False,False,True
1,False,False,False
2,False,False,True
3,True,False,False
4,False,False,False
5,False,True,True


In [69]:
Df.notnull()

,Int,Float,Complex
0,True,True,False
1,True,True,True
2,True,True,False
3,False,True,True
4,True,True,True
5,True,False,False


In [70]:
Df.notna()

,Int,Float,Complex
0,True,True,False
1,True,True,True
2,True,True,False
3,False,True,True
4,True,True,True
5,True,False,False


In [71]:
Fill = np.nansum(Df['Int']).astype(float)
Fill

170.0

In [72]:
Fill_Missing_Value = Df.fillna(Fill,axis=1)
Fill_Missing_Value

,Int,Float,Complex
0,10.0,20.5,170.000000+0.000000j
1,20.0,10.5,3.000000+2.000000j
2,30.0,5.5,170.000000+0.000000j
3,170.0,2.5,2.000000+23.000000j
4,50.0,6.5,2.000000+5.000000j
5,60.0,170.0,170.000000+0.000000j


In [73]:
for i in Df.columns:
    fill = np.nanmean(Df[i])
    Df[i].replace(np.nan,fill,inplace=True)
print(Df)

    Int  Float              Complex
0  10.0   20.5  2.333333+10.000000j
1  20.0   10.5   3.000000+2.000000j
2  30.0    5.5  2.333333+10.000000j
3  34.0    2.5  2.000000+23.000000j
4  50.0    6.5   2.000000+5.000000j
5  60.0    9.1  2.333333+10.000000j



Here the first and last row have been dropped, because they contain only two non-null values.

### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [75]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill NA entries with a single value, such as zero:

In [76]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:

In [78]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [79]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For ``DataFrame``s, the options are similar, but we can also specify an ``axis`` along which the fills take place:

In [80]:
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [81]:
df.fillna(method='ffill', axis=1) # row 

,A,B,C,D
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,9.0,9.0


In [82]:
df.fillna(method='bfill', axis=1)

,A,B,C,D
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,9.0,NaN


In [83]:
Data = pd.read_csv('data/california_cities.csv')
Data

,Unnamed: 0,city,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
0,0,Adelanto,34.576111,-117.432778,875.0,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
1,1,AgouraHills,34.153333,-118.761667,281.0,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
2,2,Alameda,37.756111,-122.274444,NaN,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79
3,3,Albany,37.886944,-122.297778,NaN,43.0,18969,5.465,1.788,3.677,14.155,4.632,9.524,67.28
4,4,Alhambra,34.081944,-118.135000,150.0,492.0,83089,7.632,7.631,0.001,19.766,19.763,0.003,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,477,Yountville,38.403056,-122.362222,30.0,98.0,2933,1.531,1.531,0.000,3.966,3.966,0.000,0.00
478,478,Yreka,41.726667,-122.637500,787.0,2582.0,7765,10.053,9.980,0.073,26.036,25.847,0.188,0.72
479,479,YubaCity,39.134722,-121.626111,18.0,59.0,64925,14.656,14.578,0.078,37.959,37.758,0.201,0.53
480,480,Yucaipa,34.030278,-117.048611,798.0,2618.0,51367,27.893,27.888,0.005,72.244,72.231,0.013,0.02


In [84]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          482 non-null    int64  
 1   city                482 non-null    object 
 2   latd                482 non-null    float64
 3   longd               482 non-null    float64
 4   elevation_m         434 non-null    float64
 5   elevation_ft        470 non-null    float64
 6   population_total    482 non-null    int64  
 7   area_total_sq_mi    480 non-null    float64
 8   area_land_sq_mi     482 non-null    float64
 9   area_water_sq_mi    481 non-null    float64
 10  area_total_km2      477 non-null    float64
 11  area_land_km2       478 non-null    float64
 12  area_water_km2      478 non-null    float64
 13  area_water_percent  477 non-null    float64
dtypes: float64(11), int64(2), object(1)
memory usage: 52.8+ KB


In [85]:
Data.describe()

,Unnamed: 0,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
count,482.000000,482.000000,482.000000,434.000000,470.000000,4.820000e+02,480.000000,482.000000,481.000000,477.000000,478.000000,478.000000,477.000000
mean,240.500000,36.095088,-119.837714,180.416705,567.168723,6.489493e+04,18.442183,16.840981,1.589630,46.753283,43.547021,3.125178,5.301801
std,139.285678,2.298006,2.147676,309.531825,1001.265722,2.032041e+05,36.363230,32.694647,9.490386,90.972269,84.933022,11.562269,14.656428
min,0.000000,32.578333,-124.263333,-34.000000,-180.000000,1.000000e+00,0.314000,0.314000,0.000000,0.813000,0.813000,0.000000,0.000000
25%,120.250000,33.977292,-121.954931,22.000000,69.000000,1.090200e+04,3.708250,3.506500,0.000000,9.551000,9.025000,0.000250,0.000000
50%,240.500000,36.204722,-119.708611,67.000000,200.000000,2.905750e+04,9.093000,8.362000,0.029000,23.551000,21.533000,0.078500,0.340000
75%,360.750000,37.886667,-117.977292,173.500000,515.000000,6.646650e+04,19.878250,19.090000,0.244000,51.772000,49.322000,0.623250,1.870000
max,481.000000,41.965000,-114.596389,2402.000000,7880.000000,3.884307e+06,503.000000,469.000000,185.020000,1302.000000,1214.000000,122.270000,84.580000


In [86]:
Data_Ch = Data.drop(['Unnamed: 0'],axis=1)

In [87]:
Data_Ch.head()

,city,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
0,Adelanto,34.576111,-117.432778,875.0,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
1,AgouraHills,34.153333,-118.761667,281.0,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
2,Alameda,37.756111,-122.274444,NaN,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79
3,Albany,37.886944,-122.297778,NaN,43.0,18969,5.465,1.788,3.677,14.155,4.632,9.524,67.28
4,Alhambra,34.081944,-118.135000,150.0,492.0,83089,7.632,7.631,0.001,19.766,19.763,0.003,0.01


In [88]:
Data_Ch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city                482 non-null    object 
 1   latd                482 non-null    float64
 2   longd               482 non-null    float64
 3   elevation_m         434 non-null    float64
 4   elevation_ft        470 non-null    float64
 5   population_total    482 non-null    int64  
 6   area_total_sq_mi    480 non-null    float64
 7   area_land_sq_mi     482 non-null    float64
 8   area_water_sq_mi    481 non-null    float64
 9   area_total_km2      477 non-null    float64
 10  area_land_km2       478 non-null    float64
 11  area_water_km2      478 non-null    float64
 12  area_water_percent  477 non-null    float64
dtypes: float64(11), int64(1), object(1)
memory usage: 49.1+ KB


In [90]:
Data_Ch.columns

Index(['city', 'latd', 'longd', 'elevation_m', 'elevation_ft',
       'population_total', 'area_total_sq_mi', 'area_land_sq_mi',
       'area_water_sq_mi', 'area_total_km2', 'area_land_km2', 'area_water_km2',
       'area_water_percent'],
      dtype='object')

In [92]:
Data_Ch.isna().sum()

city                   0
latd                   0
longd                  0
elevation_m           48
elevation_ft          12
population_total       0
area_total_sq_mi       2
area_land_sq_mi        0
area_water_sq_mi       1
area_total_km2         5
area_land_km2          4
area_water_km2         4
area_water_percent     5
dtype: int64

In [93]:
fill = np.nanmean(Data_Ch['elevation_m'])
fill

180.41670506912445

In [94]:
Data_Ch['elevation_m']=Data_Ch['elevation_m'].fillna(fill)

In [95]:
Data_Ch

,city,latd,longd,elevation_m,elevation_ft,population_total,area_total_sq_mi,area_land_sq_mi,area_water_sq_mi,area_total_km2,area_land_km2,area_water_km2,area_water_percent
0,Adelanto,34.576111,-117.432778,875.000000,2871.0,31765,56.027,56.009,0.018,145.107,145.062,0.046,0.03
1,AgouraHills,34.153333,-118.761667,281.000000,922.0,20330,7.822,7.793,0.029,20.260,20.184,0.076,0.37
2,Alameda,37.756111,-122.274444,180.416705,33.0,75467,22.960,10.611,12.349,59.465,27.482,31.983,53.79
3,Albany,37.886944,-122.297778,180.416705,43.0,18969,5.465,1.788,3.677,14.155,4.632,9.524,67.28
4,Alhambra,34.081944,-118.135000,150.000000,492.0,83089,7.632,7.631,0.001,19.766,19.763,0.003,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,Yountville,38.403056,-122.362222,30.000000,98.0,2933,1.531,1.531,0.000,3.966,3.966,0.000,0.00
478,Yreka,41.726667,-122.637500,787.000000,2582.0,7765,10.053,9.980,0.073,26.036,25.847,0.188,0.72
479,YubaCity,39.134722,-121.626111,18.000000,59.0,64925,14.656,14.578,0.078,37.959,37.758,0.201,0.53
480,Yucaipa,34.030278,-117.048611,798.000000,2618.0,51367,27.893,27.888,0.005,72.244,72.231,0.013,0.02


In [96]:
Data_Ch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city                482 non-null    object 
 1   latd                482 non-null    float64
 2   longd               482 non-null    float64
 3   elevation_m         482 non-null    float64
 4   elevation_ft        470 non-null    float64
 5   population_total    482 non-null    int64  
 6   area_total_sq_mi    480 non-null    float64
 7   area_land_sq_mi     482 non-null    float64
 8   area_water_sq_mi    481 non-null    float64
 9   area_total_km2      477 non-null    float64
 10  area_land_km2       478 non-null    float64
 11  area_water_km2      478 non-null    float64
 12  area_water_percent  477 non-null    float64
dtypes: float64(11), int64(1), object(1)
memory usage: 49.1+ KB


In [97]:
Data_Ch['elevation_ft']=Data_Ch['elevation_ft'].fillna(np.nanmean(Data_Ch['elevation_ft']))

In [98]:
Data_Ch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city                482 non-null    object 
 1   latd                482 non-null    float64
 2   longd               482 non-null    float64
 3   elevation_m         482 non-null    float64
 4   elevation_ft        482 non-null    float64
 5   population_total    482 non-null    int64  
 6   area_total_sq_mi    480 non-null    float64
 7   area_land_sq_mi     482 non-null    float64
 8   area_water_sq_mi    481 non-null    float64
 9   area_total_km2      477 non-null    float64
 10  area_land_km2       478 non-null    float64
 11  area_water_km2      478 non-null    float64
 12  area_water_percent  477 non-null    float64
dtypes: float64(11), int64(1), object(1)
memory usage: 49.1+ KB


Notice that if a previous value is not available during a forward fill, the NA value remains.

<!--NAVIGATION-->
< [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Contents](Index.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) >